# [Day 5](https://adventofcode.com/2023/day/5)
## Part 1

In [1]:
from dataclasses import dataclass

def isDigit(c):
    return c >= '0' and c <= '9'

@dataclass
class Range:
    dest: int
    source: int
    length: int
    def __init__(self, line):
        (self.dest, self.source, self.length)  = [int(n) for n in line.split(' ')]
        

@dataclass
class Rule:
    ranges: list[Range]
        

def readInput(filename) -> tuple[list[int], list[Rule]]:
    output: list[Rule] = []
    seeds: list[int] = []

    with open(filename, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            if len(line):
                
                if isDigit(line[0]):
                    output[-1].ranges.append(Range(line))
                elif line.startswith('seeds:'):
                    seeds = [int(n) for n in line[6::].strip().split(' ')]
                else:
                    output.append(Rule([]))

    return (sorted(seeds), output)

In [2]:
import bisect

(seeds, rules) = readInput('input.txt')

for rule in rules:

    newSeeds = list()
    for r in rule.ranges:

        i = bisect.bisect_left(seeds, r.source)
        if i >= len(seeds):
            continue
        
        j = bisect.bisect_right(seeds, r.source + r.length - 1)
        if i == j:
            continue
        newSeeds += [s - r.source + r.dest for s in seeds[i:j]]
        seeds = seeds[:i] + seeds[j:]
        
    newSeeds += seeds
    seeds = sorted(newSeeds)

print(seeds[0])


165788812


## Part 2

In [3]:
@dataclass
class SeedRange:
    start: int
    length: int

    def __lt__(self, other):
        return self.start < other.start
    
def readInput2(filename) -> tuple[list[Range], list[Rule]]:
    output: list[Rule] = []
    seeds: list[Range] = []

    with open(filename, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            if len(line):
                
                if isDigit(line[0]):
                    output[-1].ranges.append(Range(line))
                elif line.startswith('seeds:'):
                    seedsLine = line[6::].strip().split(' ')
                    seeds = [SeedRange(int(seedsLine[i]), int(seedsLine[i+1])) for i in range(0, len(seedsLine), 2)]
                else:
                    output.append(Rule([]))

    return (sorted(seeds), output)


In [4]:
import bisect

(seeds, rules) = readInput2('input.txt')

for rule in rules:
    newSeeds = list()
    for r in rule.ranges:

        i = bisect.bisect_left(seeds, r.source, key=lambda x: x.start + x.length)
        if i >= len(seeds):
            continue
        
        j = bisect.bisect_right(seeds, r.source + r.length - 1, key=lambda x: x.start)
        if i == j:
            continue

        
        ops = list()
        # split the seeds 
        for si, seed in enumerate(seeds[i:j]):
            if r.source <= seed.start and r.source + r.length >= seed.start + seed.length:
                # seed is fully covered by the range
                seed.start = start=seed.start - r.source + r.dest
                newSeeds.append(seed)
                # remove the seed from the list
                ops.append(('r', i+si))
            elif r.source <= seed.start:
                # initial part of seed is covered by the range
                newSeeds.append(SeedRange(start=seed.start - r.source + r.dest, length = r.source + r.length - seed.start))
                # adjust the seed the remaining part
                newStart = r.source + r.length

                seed.length = seed.length - (newStart - seed.start)
                seed.start = newStart
                
            elif r.source + r.length >= seed.start + seed.length:
                # end part of seed is covered by the range
                newSeeds.append(SeedRange(start=r.dest, length = seed.start + seed.length - r.source))
                # adjust the seed length for the remaining part
                seed.length = r.source - seed.start
            else:
                # range is inside the seed, remove the inner part
                newSeeds.append(SeedRange(start=r.dest, length = r.length))
                # insert the seed end part
                ops.append(('i', i+si, SeedRange(start=r.source + r.length, length = seed.start + seed.length - (r.source + r.length))))
                # adjust the seed length for the initial part
                seed.length = r.source - seed.start

        for op in reversed(ops):
            match op[0]:
                case 'r':
                    del seeds[op[1]]
                case 'i':
                    seeds.insert(op[1], op[2])

    newSeeds += seeds
    seeds = sorted(newSeeds)

print(seeds[0].start)

1928058
